<a href="https://colab.research.google.com/github/figgarnold/Eurovision/blob/main/AudioFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

Installations:

In [1]:
# System packages needed for Essentia
!apt-get install -y build-essential libfftw3-dev libeigen3-dev libsamplerate0-dev

# Clone Essentia and build it
!git clone https://github.com/MTG/essentia.git
%cd essentia
!./waf configure --with-python
!./waf build
!./waf install

# Add Essentia to Python path
import sys
sys.path.append("/usr/local/lib/python3.8/dist-packages")  # adjust if needed

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libfftw3-dev is already the newest version (3.3.8-2ubuntu8).
libsamplerate0-dev is already the newest version (0.2.2-1build1).
libeigen3-dev is already the newest version (3.4.0-2ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
fatal: destination path 'essentia' already exists and is not an empty directory.
/content/essentia
/content/essentia/wscript:225: SyntaxWarning: invalid escape sequence '\w'
  # libs_paths = [';packaging\win32_3rdparty\\' + lib + '\lib\pkgconfig' for lib in libs_3rdparty]
Setting top to                           : /content/essentia 
Setting out to                           : /content/essentia/build 
→ configuring the project in /content/essentia
→ Building in release mode
Checking for 'g++' (C++ compiler)        : /usr/bin/g++ 
Checking for 'gcc' (C compiler)          : /usr/bin/gcc

In [2]:
!pip install Essentia

In [3]:
!pip install yt_dlp
!apt-get install -y nodejs



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nodejs is already the newest version (12.22.9~dfsg-1ubuntu3.6).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


Imports:

In [4]:
import yt_dlp
import pandas as pd
import essentia.standard as es
import os
import glob

In [31]:
print(os.getcwd())

/content/essentia


In [9]:
df = pd.read_csv("/content/translatedlyrics.csv")

# Cookies

In [41]:
from google.colab import files
uploaded = files.upload()  # choose cookies.txt

Saving cookies.txt to cookies (1).txt


# Helper Functions

Download Helper:

In [38]:
def download_audio(youtube_url, base_name, cookies_file="cookies.txt", cookie_from_browser=None):
    """
    Download audio from YouTube as WAV using yt-dlp.
    Always returns the correct .wav filename.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'/content/audio/%{base_name}.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'extractor_args': {'youtube': {'player_client': ['default']}},
        'quiet': True,
        'ignoreerrors': True
    }
    if cookies_file:
        ydl_opts['cookiefile'] = cookies_file
    if cookie_from_browser:
        ydl_opts['cookiefrombrowser'] = cookie_from_browser

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        final_path = ydl.prepare_filename(info)
        # force extension to .wav
        final_path = os.path.splitext(final_path)[0] + ".wav"
        return final_path

Feature Helper:

In [29]:
def extract_essentia_features(audio_file):
    audio = es.MonoLoader(filename=audio_file)()

    danceability = es.Danceability()(audio)
    bpm, _, _, _, rhythm_strength = es.RhythmExtractor2013(method="multifeature")(audio)

    tonal_features = es.TonalExtractor()(audio)
    key = tonal_features[0]
    scale = tonal_features[1]

    return {
        "danceability": danceability,
        "bpm": bpm,
        "rhythm_strength": rhythm_strength,
        "key": key,
        "scale": scale
    }


# Pipeline

Batch Run:

In [42]:
results = []
batch_size = 20
cookies_path = "cookies.txt"  # optional, if uploaded

# resume at a specific index if needed
start_index = 145

for start in range(start_index, len(df), batch_size):
    batch = df.iloc[start:start+batch_size]
    print(f"Processing batch {start}–{start+len(batch)-1}")

    for idx, row in batch.iterrows():
        url = row["youtube_url"]
        title = str(row.get("title", f"track_{idx}"))

        try:
            audio_path = download_audio(url, base_name=title, cookies_file=cookies_path)

            if not os.path.exists(audio_path):
                print(f"✗ No audio file created for row {idx} (title='{title}')")
                continue

            feats = extract_essentia_features(audio_path)
            feats["title"] = title
            feats["youtube_url"] = url
            results.append(feats)

            os.remove(audio_path)  # free disk space
            print(f"✓ {title} processed (index {idx})")

        except Exception as e:
            print(f"Error with {title} (index {idx}): {e}")
            continue

    # Save after each batch
    pd.DataFrame(results).to_csv("features_partial.csv", mode="a", header=False, index=False)
    results = []  # clear memory before next batch
    print(f"✔ Batch {start}–{start+len(batch)-1} saved")

Processing batch 145–164


ERROR: [youtube] 4wQ_sRUCfpM: Video unavailable. This content isn't available, try again later. The current session has been rate-limited by YouTube for up to an hour. It is recommended to use `-t sleep` to add a delay between video requests to avoid exceeding the rate limit. For more information, refer to  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#this-content-isnt-available-try-again-later


Error with track_145 (index 145): 'NoneType' object has no attribute 'setdefault'


ERROR: [youtube] qfKk0uEGsPA: Video unavailable. This content isn't available, try again later. The current session has been rate-limited by YouTube for up to an hour. It is recommended to use `-t sleep` to add a delay between video requests to avoid exceeding the rate limit. For more information, refer to  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#this-content-isnt-available-try-again-later


Error with track_146 (index 146): 'NoneType' object has no attribute 'setdefault'


...

ERROR: [youtube] reJAq87Wwrw: Video unavailable. This content isn't available, try again later. The current session has been rate-limited by YouTube for up to an hour. It is recommended to use `-t sleep` to add a delay between video requests to avoid exceeding the rate limit. For more information, refer to  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#this-content-isnt-available-try-again-later


Error with track_200 (index 200): 'NoneType' object has no attribute 'setdefault'


ERROR: [youtube] -wXJGlSmquQ: Video unavailable


Error with track_201 (index 201): 'NoneType' object has no attribute 'setdefault'


KeyboardInterrupt: 

# Consolidate

Merge features into original data frame and output new csv:

In [ ]:
all_files = glob.glob("features_partial*.csv")
dfs = [pd.read_csv(f) for f in all_files]
final_df = pd.concat(dfs, ignore_index=True)

final_df.to_csv("features_all.csv", index=False)
print(final_df.head())

In [ ]:
lyrics_df = pd.read_csv("translatedlyrics.csv")
features_df = pd.read_csv("features_all.csv")

print(lyrics_df.columns)
print(features_df.columns)


In [ ]:
merged_df = pd.merge(
    lyrics_df,
    features_df,
    on=["title", "youtube_url"],   # join on both to be safe
    how="left"                     # keep all lyrics, even if audio missing
)

In [ ]:
merged_df.to_csv("lyrics_audio_merged.csv", index=False)
print(merged_df.head())